In [ ]:
import numpy as np
import pandas as pd
from gensim.models import word2vec, KeyedVectors, TfidfModel
import MeCab
import neologdn
import re
import emoji
from sklearn.cluster import KMeans
from collections import defaultdict
import collections
from gensim import corpora
from operator import itemgetter

### モデル構築

In [ ]:
tagger = MeCab.Tagger('') # NEologd辞書指定

tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [ ]:
# 学習データの読み込み
df = pd.read_csv('') # レビュー格納csvを指定
df_bakery = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)

In [ ]:
# 不要な単語を除く
def remove_unnecessary(text):
    # 半角と全角の統一と重ね表現の除去
    normalized_text = neologdn.normalize(text)
    # URLの除去
    text_without_url = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', normalized_text)
    # 絵文字の除去
    text_without_emoji = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text_without_url])
    # 桁区切りの除去と数字の置換
    tmp = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text_without_emoji)
    text_replaced_number = re.sub(r'\d+', '0', tmp)
    # 半角記号の置換
    tmp = re.sub(r'[!-/:-@[-`{-~]', r' ', text_replaced_number)
    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
    text_removed_symbol = re.sub(u'[■-♯]', ' ', tmp)
    return(text_removed_symbol)

In [ ]:
# コーパス作成
wakati_bakery_text = []
for i in df_bakery['review']:
    text = remove_unnecessary(i)
    wakati = tokenize(text, 2, 10000, True)
    wakati_bakery_text.append(wakati)

wakati_bakery_list = np.asarray(wakati_bakery_text, dtype = object) 
np.savetxt(" ", wakati_bakery_list, fmt = '%s', delimiter = ',', encoding = 'utf-8') # コーパス保存ディレクトリ指定

# モデル作成
word2vec_bakery_model3 = word2vec.Word2Vec(wakati_bakery_text, sg = 1, vector_size = 100, window = 5, min_count = 5, epochs = 100, workers = 3)

#sg（0: CBOW, 1: skip-gram）,vector_size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,epochs（トレーニング反復回数）

# モデルのセーブ
word2vec_bakery_model3.save(" ") # word2vecモデル保存ディレクトリ指定

### 単語分類

In [ ]:
# 単語の頻出度をカウント
# 辞書のキー値を変更
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)

# ベーカリーワード作成
def make_bakery_word():

    # モデル読み込み
    model = KeyedVectors.load(' ') # word2vecモデル保存ディレクトリ指定

    # wordとvectorのリスト
    max_vocab = 40000
    vocab = list(model.wv.index_to_key)[:max_vocab]
    vectors = [model.wv[word] for word in vocab]

    # k-meansクラスタリング
    n_clusters = 6 #クラスター数はこちらで任意の値を定める
    kmeans_model = KMeans(n_clusters=n_clusters, verbose=0, random_state=42)
    kmeans_model.fit(vectors)

    # クラスタ辞書化
    cluster_labels = kmeans_model.labels_
    cluster_to_words = defaultdict(list)
    for cluster_id, word in zip(cluster_labels, vocab):
        cluster_to_words[cluster_id].append(word)
        
    for words in cluster_to_words.values():
        print(words[:60])

    # 辞書のキー値を変更
    change_dict_key(cluster_to_words, 0, 'その他1')
    change_dict_key(cluster_to_words, 1, 'パンの材料')
    change_dict_key(cluster_to_words, 2, 'その他2')
    change_dict_key(cluster_to_words, 3, 'その他3')
    change_dict_key(cluster_to_words, 4, 'パンの種類')
    change_dict_key(cluster_to_words, 5, 'その他4')

    df_dict = pd.DataFrame.from_dict(cluster_to_words, orient="index").T
    print(df_dict.iloc[:,[0,1,2,3,4,5]])


    # 抽出したい分類のみbakery_wordに入れる
    value_words = cluster_to_words['パンの種類']
    taste_words = cluster_to_words['パンの材料']
    value_words.extend(taste_words)
    bakery_word = value_words

    # いらないワードを削除
    delete_words = {'生地', '使用', '風味', 'タイプ', '風味', 'リベイク', '上品', '個人的', 
                    'ちり', '丸く', 'トロ', '熱々', '個人的', 'お水', '香り', '見た目', 
                    '食感', '大人', 'パリ', 'わり', 'パン生地', '表面', 'テクスチャ', '旨い',
                    '寄り', 'サイズ', '好み', '相方', '絶賛', '税抜','っぽい', 'リベイク',
                    'たっぷり', 'ぱん', '税込', '香ばしく', '味わい', '良き', 'in', '焦げ', '硬さ',
                    'チョイス', '欲しかっ', '税別', '夕食', 'よい', '水分', '断面', '食べ応え','糖質',
                    '心地', 'お供', '具材', 'リッチ', '次回', '控えめ', '繊細', 'get', '雑穀', '優しい',
                    '比率', 'プラス', '田舎', 'たより', '良し', '補給', 'どっち', 'タイプ', '小さい', 'バリ',
                    'etc', '主張', 'きめ細かく', '当日', '定番', '相性', 'それら', '共演',
                    '翌朝', '外側', '豊か', '一口', '意外', '表現', '全体的', 'わり', '出来立て','満点',
                    'no', 'チョイス', '控えめ', '味付け', 'モソモソ', '工夫', 'ベーシック', 'シンプル', 'cube',
                    'アクセント', 'かつ', '懐かし', '満足感', '三角', '苦手', '特製', '好い', 'どちら', '密度',
                    'サイズ', '提供', '両方', '季節','中身', '単品', '食後', '狙い', '限定','それぞれ', '都度', 
                    '自体', '甘酸っぱい', '〃∇〃', 'お薦め', '特徴', '少量', '空気', '小腹','めん', '形状',
                    'フカフカ', '好物', 'リベイク', '少なめ', '歯切れ', 'っぽく', '程よく', 'ひとくち', '存在感',
                    'ふか', '盛り', '入り', '同時', '共演', '旨い', 'プラス', 'たっぷり', 'ミニ','単品',
                    'お水', '柔らかい', '柔らか', 'うま', 'サイズ', 'そのままで', '限定', 'インパクト', 'www',
                    '個数', 'そのままで', '長時間', 'うまく', 'あなた', 'ならでは', 'そのもの', '選択', 
                    '人受け', '健康志向', '付き', '狙い', '珍しい', '豪華', '瀬戸内', '期待外れ', '苦味',
                    'お供', '軽い', 'in', 'ばっち', 'sns', '入り', '見た目', 'サクッ', '漬け', 'メチャ', 'かた',
                    '美味', 'あか', '新作', '半分', 'ずぶ', '飽き', '絡み', 'イケ', '気泡', '食欲', 'オヤツ', 'no',
                    'わたし', '文句', '適度', '驚き', 'かた', '適当', '温かい', '満点', '独特', 'パンチ', 'オイリー',
                    'とおり', '重視', 'ボロボロ', '0g', '中間', '絶品', 'good', '苦手', 'no', '飽き', 'コシ','薄く',
                    '瞬間', '強かっ', '期待外れ', '系統', '注入', '組み合わせ', '惜しい', 'ぅな', '控え目', '好物',
                    '残り', '単調', '基本', 'かな', 'best', 'much', 'ふか', 'しつこ', 'がち', '無難', '加減',
                    '現金', '切り', 'ツレ', 'あら', 'paypay', '空洞', '讃岐', '強く', '許容範囲', '口当たり',
                    'びき', 'セール', 'しぃ', '覚醒', 'そのままで', 'よし', '重い', '帰宅', 'お酒', 'クセ', '向き', '強い',
                    '原材料', '庶民'}
    
    for word in delete_words:
        if word in bakery_word:
            bakery_word.remove(word)

    # 必要なワードを追加
    add_words = ['食パン']
    bakery_word.extend(add_words)

    # ベーカリーワードreturn
    return bakery_word

# 頻出トップ20作成
def make_text_top20(review_words):
    top20 = []
    for words in review_words:
        c = collections.Counter(words)
        c = c.most_common(20)
        top20.append([i[0] for i in c])
    return top20

# tfidfトップ20作成
def make_tfidf_top20(review_words):
    trainings = review_words[:]
    
    # 単語->id変換の辞書作成
    dictionary = corpora.Dictionary(trainings)

    # textをcorpus化
    corpus = list(map(dictionary.doc2bow, trainings))

    # tfidf modelの生成
    test_model = TfidfModel(corpus)

    # corpusへのモデル適用
    corpus_tfidf = test_model[corpus]

    # id->単語へ変換
    tfidf = [] # id -> 単語表示に変えた文書ごとのTF-IDF
    for doc in corpus_tfidf:
        words = []
        for word in doc:
            words.append([dictionary[word[0]], word[1]])
        tfidf.append(words)

    #TF-IDF値を高い順に並び替え上位単語20個に絞る。
    top20 = [] 
    for i in range(len(tfidf)):
        soted = sorted(tfidf[i], key=itemgetter(1), reverse=True)
        soted_top20 = soted[:20]
        word_list = []
        for k in range(len(soted_top20)):
            word = soted_top20[k][0]
            word_list.append(word)
        top20.append(word_list)

    return top20

In [ ]:
if __name__ == '__main__':

    # ベーカリーワード作成
    bakery_word = make_bakery_word()

    # レビューのワードをbakery_wordでフィルタリング
    review_words = []
    with open(' ','r', encoding="utf-8") as f: # コーパスファイル指定
        for data in f:
            word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','').split(",")
            review_words.append([i for i in word if i in bakery_word])

    # 頻出トップ20作成
    text_top20 = make_text_top20(review_words)

    # tfidfトップ20作成
    tfidf_top20 = make_tfidf_top20(review_words)

    # 結果をデータフレームにしてcsvに書き出す
    df = pd.read_csv(' ') # レビュー格納csvを指定
    df_bakery = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(''.join).reset_index().sort_values('score', ascending=False)
    df_bakery['text_top20'] = text_top20
    df_bakery['tfidf_top20'] = tfidf_top20
    df_bakery['id'] = ['ID-' + str(i + 1).zfill(3) for i in range(len(df_bakery.index))]
    df_bakery_sorted_top20 = df_bakery.iloc[:,[6,0,1,2,4,5]].reset_index(drop=True)
    df_bakery_sorted_top20.to_csv("out_top20.csv", index = False)